### Dataset & DataLoader 살펴보기
- Pytorch에서 배치크기만 데이터를 조절하기 위한 메커니즘
- Dataset : 사용 데이터를 기반으로 사용자정의 클래스 작성
- DataLoader : 지정된 Dataset에서 지정된 batch size만큼 피쳐와 타겟을 추출하여 전달

[1] 모듈 로딩 및 데이터 준비 <hr>

In [298]:
### ===> 모듈 로딩
import torch 
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

import numpy as np
import pandas as pd


In [299]:
### ===> 데이터 준비
x_data=torch.IntTensor([[10,20,30],[20,30,40],[30,40,50],[40,50,60],[50,60,70]])
y_data=torch.IntTensor([[20],[30],[40],[50],[60]])  #개수가 맞아야 줄수 있다.(5==5)

print(f'x_data: {x_data.shape},{x_data.ndim}')
print(f'y_data: {y_data.shape},{y_data.ndim}')

x_data: torch.Size([5, 3]),2
y_data: torch.Size([5, 1]),2


[2] 데이터셋 생성 <hr>

- ### [2-1] TensorDataset 활용: Dataset의 sub_class(자식클래스) 

In [300]:
# TensorDataset 클래스 로딩
from torch.utils.data import TensorDataset

In [301]:
dataset=TensorDataset(x_data,y_data)
dataset

In [302]:
dataset.tensors

(tensor([[10, 20, 30],
         [20, 30, 40],
         [30, 40, 50],
         [40, 50, 60],
         [50, 60, 70]], dtype=torch.int32),
 tensor([[20],
         [30],
         [40],
         [50],
         [60]], dtype=torch.int32))

In [303]:
## __getitem__() 메서드 호출
dataset[0]

(tensor([10, 20, 30], dtype=torch.int32), tensor([20], dtype=torch.int32))

In [304]:
len(dataset)

5

- ### [2-2] 사용자 정의 데이터셋 생성 

In [305]:
### 데이터 준비
filename= '../data/irisno.csv'

irisDF=pd.read_csv(filename,header=None,delimiter=',')
irisDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       150 non-null    float64
 1   1       150 non-null    float64
 2   2       150 non-null    float64
 3   3       150 non-null    float64
 4   4       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [306]:
irisNP=np.loadtxt(filename,delimiter=',',usecols=[0,1,2,3])

In [307]:
irisDF

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [308]:

### 사용자정의 DataSet 클래스
#- 데이터의 텐서 변환
class DLDataset(Dataset):
    
    # 초기화 함수 
    def __init__(self,x_data,y_data):
        super().__init__()
        #x,y 데이터 ==> ndarray
        x_data=x_data.values if isinstance(x_data,pd.DataFrame) else x_data #values는 넘파이로 바꾸어 주는 친구
        y_data=y_data.values if isinstance(y_data,pd.DataFrame) else y_data
        
        #ndarray ==> tensor(데이터 타입이 뭐가 들어올지 몰라서 tensor를 씀)
        self.feature=torch.FloatTensor(x_data)
        self.target=torch.FloatTensor(y_data)
        
    #데이터셋의 갯수 체크 함수
    def __len__(self):
        return len(self.target)
        #return self.target.shape[0]
    
    #특정 인덱스 데이터+라벨 반환 콜백함수(callback function)
    def __getitem__(self,index):
        return self.feature[index],self.target[index]  #튜플로 반환 

In [309]:
## 데이터의 타입 체크( 조건문 만들 때 필요)
type(irisDF),type(irisNP),irisDF.__class__.__name__,irisNP.__class__.__name__

(pandas.core.frame.DataFrame, numpy.ndarray, 'DataFrame', 'ndarray')

In [310]:
if irisDF.__class__.__name__=='DataFrame':
    print('DF')
else:
    print('------')
    

DF


In [311]:
isinstance(irisDF,pd.DataFrame),isinstance(irisNP,pd.DataFrame)

(True, False)

In [312]:
isinstance([10],list),isinstance({'A':22},list)

(True, False)

In [313]:
## 데이터를 feature와 라벨로 분리
featureDF=irisDF[irisDF.columns[:-1]]
targetDF=irisDF[irisDF.columns[-1]].to_frame()  #데이터 프레임화 해서 2차원으로 변경

print(f'featureDF=> {featureDF.shape},{featureDF.ndim}D')
print(f'targetDF=> {targetDF.shape},{targetDF.ndim}D')

featureDF=> (150, 4),2D
targetDF=> (150, 1),2D


In [314]:
#object 타입 타겟 ====> int 타입 타겟 변환
from sklearn.preprocessing import LabelEncoder


targetNP=LabelEncoder().fit_transform(targetDF)
targetNP=targetNP.reshape(-1,1) # reshape(-1,1) 뒤에 나오는 1은 행에 있어야 할 개수 그러니까 1을 준건 1개, 밑에 1개 이런식으로 띄워서 쓰기 위한것 그리고 앞에 -1은 그렇게 한줄로 알아서 맞춤으로 만들라는 뜻
#10개라면 ....reshape(-1,2) => 원소,원소
#                             원소,원소
#                             원소,원소


print(f'targetNP=>{targetNP.shape},{targetNP.ndim}D')

targetNP=>(150, 1),2D


C:\ProgramData\anaconda3\envs\Torch_PY38\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [315]:
# 데이터셋 생성===> DF,NP
my_dataset=DLDataset(featureDF,targetNP)
my_dataset[0],featureDF.iloc[0],targetDF.iloc[0]

((tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0.])),
 0    5.1
 1    3.5
 2    1.4
 3    0.2
 Name: 0, dtype: float64,
 4    Iris-setosa
 Name: 0, dtype: object)

In [316]:
# 데이터셋 생성===> NP,NP
my_dataset2=DLDataset(irisNP,targetNP)
my_dataset2[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0.]))

### [2-3] 학습용, 검증용, 테스트용(3개로 나누기) Dataset <hr> 
 

In [317]:
from torch.utils.data import random_split

#학습용,검증용,테스트 데이터 비율
seed=torch.Generator().manual_seed(42)
trainDS,validDS,testDS=random_split(my_dataset2,[0.7,0.1,0.2],generator=seed) # 비율로 주기
#      학습용                           검증용                      테스트용
print(f'trainDS=>{len(trainDS)}개,validDS={len(validDS)}개,testDS={len(testDS)}개')
print('--------------------------------------------------------------------------')
print(f'Subset 속성 => indices : {trainDS.indices} \ndataset : {trainDS.dataset}')
print('-------------------------------------------------------------------------')
print(f'Subset 속성 => indices : {validDS.indices} \ndataset : {validDS.dataset}')
print('-------------------------------------------------------------------------')
print(f'Subset 속성 => indices : {testDS.indices} \ndataset : {testDS.dataset}')


trainDS=>105개,validDS=15개,testDS=30개
--------------------------------------------------------------------------
Subset 속성 => indices : [42, 95, 30, 64, 52, 35, 130, 40, 82, 17, 108, 94, 68, 97, 117, 127, 41, 44, 57, 140, 149, 32, 23, 102, 16, 113, 71, 18, 67, 66, 0, 25, 101, 112, 91, 3, 59, 116, 86, 84, 106, 142, 43, 39, 26, 98, 93, 20, 87, 19, 120, 114, 7, 63, 76, 89, 36, 45, 37, 56, 58, 122, 51, 145, 24, 21, 105, 62, 15, 11, 48, 133, 88, 50, 6, 134, 111, 8, 49, 75, 69, 124, 4, 147, 80, 100, 99, 141, 47, 107, 13, 109, 129, 28, 38, 53, 121, 5, 55, 31, 73, 74, 54, 29, 12] 
dataset : <__main__.DLDataset object at 0x000002A4E5D81E50>
-------------------------------------------------------------------------
Subset 속성 => indices : [22, 104, 81, 1, 103, 125, 85, 2, 96, 128, 27, 118, 77, 110, 146] 
dataset : <__main__.DLDataset object at 0x000002A4E5D81E50>
-------------------------------------------------------------------------
Subset 속성 => indices : [72, 139, 131, 60, 65, 92, 135, 83, 14, 

[3] DataLoader 생성 : 학습용,검증용,테스트용 <hr> 

In [318]:
#DataLoader 생성
# drop_last 매개변수: 배치사이즈로 데이터셋 분리 후 남는 데이터 처리 방법 설정 [기본: False]
batch=10
trainDL=DataLoader(trainDS,batch_size=batch)
validDL=DataLoader(validDS,batch_size=batch)
testDL=DataLoader(testDS,batch_size=batch)

len(trainDL),len(validDL),len(testDL)  #105/batch   , 15/batch  ,30/batch 

(11, 2, 3)

In [319]:
# Epoch 당 반복 단위
print(f'batch_size:{batch}')
print(f'trainDS=>{len(trainDS)}개,validDF=>{len(validDS)}개,testDS=>{len(testDS)}개')
print(f'trainDL=>{len(trainDL)}개,validDL=>{len(validDL)}개,testDL=>{len(testDL)}개')

batch_size:10
trainDS=>105개,validDF=>15개,testDS=>30개
trainDL=>11개,validDL=>2개,testDL=>3개


In [320]:
# DataLoader 속성
for _,(feature,target) in enumerate(trainDL):
    print(f'[{_} feature{feature.shape}')
    ## 로더에서 가지고온 데이터 만큼 학습 진행 => 검증 진행

[0 featuretorch.Size([10, 4])
[1 featuretorch.Size([10, 4])
[2 featuretorch.Size([10, 4])
[3 featuretorch.Size([10, 4])
[4 featuretorch.Size([10, 4])
[5 featuretorch.Size([10, 4])
[6 featuretorch.Size([10, 4])
[7 featuretorch.Size([10, 4])
[8 featuretorch.Size([10, 4])
[9 featuretorch.Size([10, 4])
[10 featuretorch.Size([5, 4])


[4] Model 클래스 정의: <hr> : 입/출력 피쳐수,층 수,은닉층의 노드수   <hr>
-구조 설계 
    * 입력층 :입력 <=피쳐 갯수,iris 4개
    * 은닉층: 마음대로 알아서 잘
    * 출력층: 출력 <=[분류]타겟 클래스 갯수[타겟으로 정할 열의 고유값 개수] [회귀]1개

In [321]:
# 여기서 부터 스스로 해볼 수 있도록 한다.>.>>..

In [322]:
#모델 클래스 정의
#클래스명:CModel
class CModel(nn.Module):
    #구성 요소 정의 함수
    def __init__(self,in_dim,out_dim) -> None:
        super().__init__()
        self.input_layer=nn.Linear(in_dim,100) #1  피쳐 개수 4개 주입
        self.relu=nn.ReLU()
        self.hidden_layer=nn.Linear(100,27)  #2
        self.output_layer=nn.Linear(27,out_dim)    #3  마지막에 꽃종류 3개 반출
    #순방향 학습 진행 함수
    def forward(self,x):
        y=self.input_layer(x)   #W1X1+W2X2+....+Wnxn+b 반환(#1:100개)를 반환,아직 activation function을 덜 넣음
        y=self.relu(y)            #relu 함수 결과 100개 반환
        y=self.hidden_layer(y)      #W1X1+W2X2+....+Wnxn+b 반환(#2:27개)를 반환
        y=self.relu(y)
        y=self.output_layer(y)    #W1X1+W2X2+....+Wnxn+b 반환(#3:3개)를 반환
        return y
        
        
    


### [5] 학습을 위한 준비:실행디바이스,모델, 최적화, 손실함수, 학습 횟수,학습 함수,평가함수,예측함수  <hr> 
        

[5-1] 실행 디바이스 설정


In [323]:
DEVICE='cuda' if torch.cuda.is_available() else 'cpu'  #얘를 상수화 하려면 튜플(tuple)로 만들어서 표현한다.

#학습 횟수
EPOCHS=50

In [324]:
#형태 보기
my_dataset2.feature.shape

torch.Size([150, 4])

In [325]:
#모델 인스턴스
IN,OUT=my_dataset2.feature.shape[1],len(np.unique(targetDF))
model=CModel(IN,OUT).to(DEVICE)
print(f'IN : {IN},OUT :{OUT}')
print(model)

IN : 4,OUT :3
CModel(
  (input_layer): Linear(in_features=4, out_features=100, bias=True)
  (relu): ReLU()
  (hidden_layer): Linear(in_features=100, out_features=27, bias=True)
  (output_layer): Linear(in_features=27, out_features=3, bias=True)
)


In [326]:

# 손실함수
LOSS_FN=nn.CrossEntropyLoss().to(DEVICE)
# 최적화
import torch.optim as optim
optimizer=optim.Adam(model.parameters()) #여기서 lr 줄수 있음

### 학습 및 검증 관련 함수 정의

[5-2] 학습진행 함수

In [327]:
def training():
    #학습 모드 => 정규화,경사하강법,드랍아웃등의 기능을 활성화
    model.train()
    #배치크기 만큼 학습 진행 및 저장
    train_loss=[]
    for cnt,(feature,target) in enumerate(trainDL):
        #배치크기만큼의 학습 데이터 준비
        feature,target=feature.to(DEVICE),target.to(DEVICE)
        target=target.squeeze
        #학습
        pre_target=model(feature)
        print(f'pre_target => {pre_target.shape},{pre_target.ndim}D')
        print(f'target => {target.shape},{target.ndim}D')
        #손실계산
        loss=LOSS_FN(pre_target,target)
        train_loss.append(loss)
   
        #w,b 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #배치 단위 학습 진행 메시지 출력
        # print(f'[Train {cnt} batch LOSS] ==> {loss}')
        
    # 에포크 단위 진행 메시지 출력
    print(f'[Train loss] ==> {loss}\n\n')
    
    return train_loss


In [328]:
for cnt,(feature,target) in enumerate(trainDL):
        print(cnt,feature,target)

0 tensor([[4.4000, 3.2000, 1.3000, 0.2000],
        [5.7000, 3.0000, 4.2000, 1.2000],
        [4.8000, 3.1000, 1.6000, 0.2000],
        [5.6000, 2.9000, 3.6000, 1.3000],
        [6.9000, 3.1000, 4.9000, 1.5000],
        [5.0000, 3.2000, 1.2000, 0.2000],
        [7.4000, 2.8000, 6.1000, 1.9000],
        [5.0000, 3.5000, 1.3000, 0.3000],
        [5.8000, 2.7000, 3.9000, 1.2000],
        [5.1000, 3.5000, 1.4000, 0.3000]]) tensor([[0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [2.],
        [0.],
        [1.],
        [0.]])
1 tensor([[6.7000, 2.5000, 5.8000, 1.8000],
        [5.6000, 2.7000, 4.2000, 1.3000],
        [6.2000, 2.2000, 4.5000, 1.5000],
        [6.2000, 2.9000, 4.3000, 1.3000],
        [7.7000, 3.8000, 6.7000, 2.2000],
        [6.1000, 3.0000, 4.9000, 1.8000],
        [4.5000, 2.3000, 1.3000, 0.3000],
        [5.1000, 3.8000, 1.9000, 0.4000],
        [4.9000, 2.4000, 3.3000, 1.0000],
        [6.7000, 3.1000, 5.6000, 2.4000]]) tensor([[2.],

[5-3] 검증 및 평가 진행함수

In [329]:
def testing():
    pass

[5-4] 예측 함수

In [330]:
def predict():
    pass

[6] 학습 진행 <hr>

In [331]:
for eps in range(EPOCHS):
    # 학습 함수
    train_loss=training()
    #검증 함수
    # testing()
    print(f'[{eps}/{EPOCHS}]{sum(train_loss)/len(train_loss)}')

pre_target => torch.Size([10, 3]),2D


AttributeError: 'builtin_function_or_method' object has no attribute 'shape'